In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from requests import get
import itertools
from tqdm.notebook import tqdm, trange
import time 
import json
from datetime import date, timedelta

In [7]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [8]:
def get_data(url):
    response = get(url, headers=headers)
    try:
        ret = json.loads(response.text)
    except ValueError:
        ret = {'dados':''}  #para resolver um bug do JSON
    return ret

In [9]:
url_1='https://dadosabertos.camara.leg.br/api/v2/deputados?pagina='
url_2='&itens=60&ordem=ASC&ordenarPor=nome'
urls=[]
for p in range(1, 10):
    urls.append(url_1+str(p)+url_2)

In [10]:
cols=['id', 'nome', 'siglaUf', 'siglaPartido', 'idLegislatura']
deputados=pd.DataFrame(columns=cols)
for url in urls:
    data = get_data(url)
    for d in data['dados']:
        data_point={}
        for c in cols:
            data_point[c]=d[c]
        deputados=deputados.append(data_point, ignore_index=True)

In [11]:
deputados.to_csv("deputados.csv", index=False)

In [12]:
def get_data_pages(url_1, url_2, cols, n_paginas=2):
    urls=[]
    for p in range(1,n_paginas+1):
        urls.append(url_1+str(p)+url_2)
    ret= pd.DataFrame(columns=cols)
    for url in tqdm(urls, leave=False):
        data =get_data(url)
        for d in data['dados']:
            data_point={}
            for c in cols:
                data_point[c]=d[c]
            ret=ret.append(data_point, ignore_index=True)
    return ret

In [13]:
cols_votacoes=['id', 'data', 'siglaOrgao', 'aprovacao']
url_1_votacoes='https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2020-01-01&dataFim=2020-02-28&pagina='
url_2_votacoes= '&ordem=DESC&ordenarPor=dataHoraRegistro'
n_pag_votacoes=10
votacoes=get_data_pages(url_1_votacoes, url_2_votacoes, cols_votacoes, n_pag_votacoes)

  0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
date_0= date(2019, 1, 1)

date_i= date_0
date_f= date_0+timedelta(days = 60)

dates=[]
for i in range (1000):
    if date_f> date.today():
        dates.append([date_i, date.today()])
        break
    if date_f.year != date_i.year:
        dates.append([date_i, date(date_i.year,12,31)])
        date_i=date(date_f.year,1,1)
        date_f=date_i+timedelta(days = 61)
    else:
        dates.append([date_i, date_f])
        date_i=date_f+timedelta(days = 1)
        date_f=date_f+timedelta(days = 62)

In [ ]:
url_1s=[]
for d in dates:
    url_1s.append('https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio='+str(d[0])+'&dataFim='+str(d[1])+'&pagina=')
cols_votacoes=['id', 'data', 'siglaOrgao', 'aprovacao']

url_2_votacoes= '&ordem=DESC&ordenarPor=dataHoraRegistro'
n_pag_votacoes=10

votacoes=pd.DataFrame(columns=cols_votacoes)

import multiprocessing
from joblib import Parallel, delayed


num_cores = multiprocessing.cpu_count()
inputs = tqdm(url_1s)

if __name__ == "__main__":
    processed_list = Parallel(n_jobs=num_cores)(delayed(get_data_pages)(i,url_2_votacoes, cols_votacoes, n_pag_votacoes) for i in inputs)

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
votacoes.to_csv("votacoes.csv", index=False)

In [ ]:
votos={}

url_1_votos='https://dadosabertos.camara.leg.br/api/v2/votacoes/'
url_2_votos='/votos' 
urls=[]
for ID in tqdm( votacoes['id']):
    url =url_1_votos+str(ID)+url_2_votos
    data =get_data(url)
    if len(data['dados'])>0:
        votos[ID]={d["deputado_"]['id']:d["tipoVoto"] for d in data['dados']}

In [ ]:
votos=pd.DataFrame(votos)
votos.to_csv("votos.csv", index=True, index_label='cod_deputado')

In [ ]:
orientacoes = {}
url_1_orientacoes = 'https://dadosabertos.camara.leg.br/api/v2/votacoes/'
url_2_orientacoes = '/orientacoes'
urls = []
for ID in tqdm(votos.columns):
    url = url_1_orientacoes+str(ID)+url_2_orientacoes
    data = get_data(url)
    if len(data['dados']) > 0:
        if d["codTipoLideranca"] == None: 
            orientacoes[ID] = {'N'+str(
                d['siglaPartidoBloco']): d["orientacaoVoto"] for d in data['dados']}
        else:
            orientacoes[ID] = {str(d["codTipoLideranca"])+str(
                d['siglaPartidoBloco']): d["orientacaoVoto"] for d in data['dados']}

In [ ]:
orientacoes=pd.DataFrame(orientacoes)
orientacoes.to_csv("orientacoes.csv", index=True, index_label='cod_partido')